In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import os
import cv2
import keras
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/machine learning/Real or fake job posting/archive/fake_job_postings.csv')
df.head(5)

In [ ]:
df.groupby('fraudulent').describe()

In [ ]:
df['fraudulent'].value_counts()

In [ ]:
df['fraudulent'].dtypes

In [ ]:
866/17014

In [ ]:
df_fake=df[df['fraudulent']==1]
df_fake.shape

In [ ]:
df_real=df[df['fraudulent']==0]
df_real.shape

In [ ]:
df_real_downsampled=df_real.sample(df_fake.shape[0])
df_real_downsampled.shape

In [ ]:
df_balanced=pd.concat([df_real_downsampled,df_fake])
df_balanced.shape

In [ ]:
df_balanced['fraudulent'].value_counts()

In [ ]:
df_balanced.dtypes

In [ ]:
df_balanced=df_balanced.drop(columns=['job_id','title','location','department','salary_range','industry','function'])

In [ ]:
df_balanced.dtypes

In [ ]:
df_balanced.isna().sum()

In [ ]:
df_balanced=df_balanced.drop(columns=['company_profile','requirements','benefits','employment_type','required_experience','required_education','telecommuting','has_company_logo','has_questions' ])
df_balanced.isna().sum()

In [ ]:
df_balanced=df_balanced.dropna(subset =['description'])

In [ ]:
df_balanced.isna().sum()

In [ ]:
df_balanced['description'] = df_balanced['description'].astype('str') 

In [ ]:
df_balanced.dtypes

In [ ]:
df_balanced.shape

In [ ]:
Y=df_balanced['fraudulent']
X=df_balanced.drop(columns=['fraudulent'])

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,stratify=Y,random_state=42)

In [ ]:
X_train.shape

In [ ]:
Y_train.value_counts()

In [ ]:
X_train.dtypes

In [ ]:
bert_preprocess_model=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder_model=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
  preprocessed_text=bert_preprocess_model(sentences)
  return bert_encoder_model(preprocessed_text)['pooled_output']

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_encoder_model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train,Y_train,epochs=23,batch_size=200)

In [ ]:
model.evaluate(X_test,Y_test)